## Youtube Data Analysis

In this notebook, we have a dataset of user comments for youtube videos related to animals or pets. We will attempt to identify cat or dog owners based on these comments, find out the topics important to them, and then identify video creators with the most viewers that are cat or dog owners.

Step 1: Identify Cat And Dog Owners Find the users who are cat and/or dog owners.

Step 2: Build And Evaluate Classifiers Build classifiers for the cat and dog owners and measure the performance of the classifiers.

Step 3: Classify All The Users Apply the cat/dog classifiers to all the users in the dataset. Estimate the fraction of all users who are cat/dog owners.

Step 4: Extract Insights About Cat And Dog Owners Find topics important to cat and dog owners.

Step 5: Identify Creators With Cat And Dog Owners In The Audience Find creators with the most cat and/or dog owners. Find creators with the highest statistically significant percentages of cat and/or dog owners.

#### 0. Data Exploration and Cleaning

In [4]:
df_clean=spark.read.csv("/FileStore/tables/animals_comments.csv",inferSchema=True,header=True)
df_clean.show(10)

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows

In [5]:
df_clean.count() 

Out[ 3 ]: 5820035

In [6]:
df_clean = df_clean.na.drop(subset=["comment"])
df_clean.count()

Out[ 4 ]: 5818984

In [7]:
df_clean.show()

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟|
Hope For Paws - O...|2911.0| That mother cat l...|
Hope For Paws - O...|2911.0| Its people like H...|
 Talking Kitty Cat|2911.0| steve: No wet foo...|
 Brave Wilderness|3224.0| Dont call this a ...|
 MaxluvsMya|3267.0| why are you alway...|
Rise Up Society F...|3372.0| Deb Tucker|
 The Dodo|3466.0| Thats a deer isnt...|
 Brave Wilderness|3466.0| there is no safe ...|
 Brave Wilderness|3466.0| Red before yellow...|
+--------------------+------+-------------------------------------+
only showing top 20 rows

In [8]:
# find user with preference of dog and cat
from pyspark.sql.functions import when
from pyspark.sql.functions import col

# you can user your ways to extract the label

df_clean = df_clean.withColumn("label", \
                           (when(col("comment").like("%my dog%"), 1) \
                           .when(col("comment").like("%I have a dog%"), 1) \
                           .when(col("comment").like("%my cat%"), 1) \
                           .when(col("comment").like("%I have a cat%"), 1) \
                           .when(col("comment").like("%my puppy%"), 1) \
                           .when(col("comment").like("%my pup%"), 1) \
                           .when(col("comment").like("%my kitty%"), 1) \
                           .when(col("comment").like("%my pussy%"), 1) \
                           .otherwise(0)))

# (large,small)=df_clean.filter(col('label')==1).randomSplit([0.99, 0.01],seed = 100)
# df_clean = small
df_clean.createOrReplaceTempView('iden_owner')

In [9]:
%sql

Select label,count(*)
From iden_owner
Group by 1


label,count(1)
1,40225
0,5778759


In [10]:
df_clean.show()

+--------------------+------+-------------------------------------+-----+
 creator_name|userid| comment|label|
+--------------------+------+-------------------------------------+-----+
 Doug The Pug| 87.0| I shared this to ...| 0|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0|
 bulletproof| 530.0| stop saying get e...| 0|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|
 ojatro|1031.0| I wanna see what ...| 0|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|
Hope For Paws - O...|1806.0| when I saw the en...| 0|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟| 0|
Hope For Paws - O...|2911.0| That mother cat l...| 0|
Hope For Paws - O...|2911.0| Its people like H...| 0|
 Talking Kitty Cat|2911.0| steve: No wet foo...| 0|
 Brave Wilderness|3224.0| Dont call this a ...| 0|
 MaxluvsMya|3267.0| why are you alway...| 0|
Rise Up Society F...|3372.0| Deb Tucker| 0|
 The Dodo|3466.0| Thats a deer isnt...| 0|
 Brave Wilderness|3466.0| there is no safe ...| 0|
 Brave Wilderness|3466.0| Red before yellow...| 0|
+--------------------+------+-------------------------------------+-----+
only showing top 20 rows

#### 1. Data preprocessing and Build the classifier

In [12]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")

remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# countvec = CountVectorizer(inputCol='filtered', outputCol='features')
word2Vec = Word2Vec(inputCol="filtered", outputCol="features",vectorSize=20)

In [13]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, remover, word2Vec])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df_clean)
dataset = pipelineFit.transform(df_clean)

In [14]:
dataset.show()

+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+
 creator_name|userid| comment|label| words| filtered| features|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+
 Doug The Pug| 87.0| I shared this to ...| 0|[i, shared, this,...|[shared, friends,...|[0.02812868240289...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0| [super, cute]| [super, cute]|[-0.0463721246924...|
 bulletproof| 530.0| stop saying get e...| 0|[stop, saying, ge...|[stop, saying, ge...|[-0.1349987564608...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|[tenho, uma, jibo...|[tenho, uma, jibo...|[0.01747478606800...|
 ojatro|1031.0| I wanna see what ...| 0|[i, wanna, see, w...|[wanna, see, happ...|[0.20113491043448...|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|[well, shit, now,...|[well, shit, im, ...|[0.08396135643124...|
Hope For Paws - O...|1806.0| when I saw the en...| 0|[when, i, saw, th...|[saw, end, said, ...|[0.01426774557330...|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|[holy, crap, that...|[holy, crap, quit...|[-0.1337657529446...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0| []| []| (20,[],[])|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|[call, the, teddy...|[call, teddy, larry]|[0.43122113247712...|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟| 0| []| []| (20,[],[])|
Hope For Paws - O...|2911.0| That mother cat l...| 0|[that, mother, ca...|[mother, cat, loo...|[0.12383166872538...|
Hope For Paws - O...|2911.0| Its people like H...| 0|[its, people, lik...|[people, like, ho...|[0.04878540150821...|
 Talking Kitty Cat|2911.0| steve: No wet foo...| 0|[steve, no, wet, ...|[steve, wet, food...|[0.16718023316934...|
 Brave Wilderness|3224.0| Dont call this a ...| 0|[dont, call, this...|[dont, call, chal...|[-0.0145847640621...|
 MaxluvsMya|3267.0| why are you alway...| 0|[why, are, you, a...|[always, awake, l...|[-0.0286065423861...|
Rise Up Society F...|3372.0| Deb Tucker| 0| [deb, tucker]| [deb, tucker]|[0.15034425072371...|
 The Dodo|3466.0| Thats a deer isnt...| 0|[thats, a, deer, ...| [thats, deer, isnt]|[-0.2134853353103...|
 Brave Wilderness|3466.0| there is no safe ...| 0|[there, is, no, s...|[safe, way, hold,...|[-0.0807316485806...|
 Brave Wilderness|3466.0| Red before yellow...| 0|[red, before, yel...|[red, yellow, kil...|[0.07578706163913...|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [15]:
(label0_train,label0_test)=dataset.filter(col('label')==1).randomSplit([0.7, 0.3],seed = 100)
(label1_train, label1_ex)=dataset.filter(col('label')==0).randomSplit([0.005, 0.995],seed = 100)
(label1_test, label1_ex2)=label1_ex.randomSplit([0.002, 0.998],seed = 100)

In [16]:
trainingData = label0_train.union(label1_train)
train1, train2 = trainingData.randomSplit([0.5, 0.5],seed = 100)
trainingData = train1
testData=label0_test.union(label1_test)

In [17]:
trainingData.createOrReplaceTempView('train')
testData.createOrReplaceTempView('test')

In [18]:
%sql

Select label,count(*)
From train
group by 1

-- view training data's number of each label

label,count(1)
1,14233
0,14294


In [19]:
%sql

Select label,count(*)
From test
group by 1

-- view test data's number of each label

label,count(1)
1,11937
0,11402


In [20]:
# print("Dataset Count: " + str(dataset.count()))
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))

##### LogisticRegression

In [22]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=8, regParam=0.05, elasticNetParam=0.8)
lrModel = lr.fit(trainingData)

print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: (20,[1,3,7,10,13,14,15,16,19],[-0.0173959045282,-0.565882328736,1.92413711826,1.7666409384,2.74801936714,-0.0971317694942,-0.142496149821,2.0711639525,-0.759259070101])
Intercept: -0.835312670264375

##### Parameter Tuning and K-fold cross-validation

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder()\
            .addGrid(lr.regParam,[0.05,0.1,0.2,0.4])\
            .addGrid(lr.maxIter,[8,10,12,14])\
            .build()
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5,
                          seed = 42)
cvModel = crossval.fit(trainingData)


/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [25]:
# best_model = cvModel.bestModel
# best_model.extractParamMap()

##### RandomForest

In [27]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='label', featuresCol='features', seed=42)
rfModel = rf.fit(trainingData)
prediction = rfModel.transform(trainingData)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(prediction)

Out[ 17 ]: 0.913907372071701

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = ParamGridBuilder()\
            .addGrid(rf.numTrees,[50,100])\
            .addGrid(rf.maxDepth,[6,8,10])\
            .build()
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5,
                          seed = 42)
cv_rfModel = crossval.fit(trainingData)


In [29]:
# best_model = cv_rfModel.bestModel
# best_model.extractParamMap()

##### Gradient boosting

In [31]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol='label', featuresCol='features', seed=42)

paramGrid = ParamGridBuilder()\
            .addGrid(gbt.stepSize,[0.05,0.1])\
            .addGrid(gbt.maxIter,[6,8,10])\
            .addGrid(gbt.maxDepth,[5,6])\
            .build()
crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5,
                          seed = 42)
cv_gbtModel = crossval.fit(trainingData)

#### Get the best model with best hyper-parameter

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

rf_predict = cv_rfModel.transform(testData)
evaluator.evaluate(rf_predict)

Out[ 21 ]: 0.9295760954095144

In [34]:
lr_predict = cvModel.transform(testData)
evaluator.evaluate(lr_predict)

Out[ 22 ]: 0.8580079732752344

In [35]:
gbt_predict = cv_gbtModel.transform(testData)
evaluator.evaluate(gbt_predict)

Out[ 23 ]: 0.9194723505795958

In [36]:
rf_predict.createOrReplaceTempView('rf')

In [37]:
%sql

Select label,prediction,count(*)
From rf
Where label!=prediction
Group by 1,2

label,prediction,count(1)
1,0.0,1099
0,1.0,2133


#### 2. Classify All The Users

In [39]:
prediction = cv_rfModel.transform(dataset)
evaluator.evaluate(prediction)

Out[ 25 ]: 0.9355269870235031

In [40]:
prediction.createOrReplaceTempView('predict')

In [41]:
%sql

Select label,prediction,count(*)
From predict
Where label!=prediction
Group by 1,2

-- find error prediction number

label,prediction,count(1)
1,0.0,2992
0,1.0,1065726


#### 3. Get insigts of Users

In [43]:
%sql

Select label,prediction,count(*)
From predict
Group by 1,2

label,prediction,count(1)
1,0.0,2992
0,0.0,4713033
1,1.0,37233
0,1.0,1065726


In [44]:
%sql

with total as
(
Select count(*) as total
From predict
),
par as
(
Select prediction,count(*) as num
From predict
Group by 1
)

Select prediction, num, num/total as frac
From par cross join total

-- have a look at total fraction

prediction,num,frac
0.0,4716025,0.8104550553842389
1.0,1102959,0.1895449446157611


#### Use CountVectorizer to transform words to vectors in order to view words in each topic

In [46]:
from pyspark.ml.feature import CountVectorizer

countvec = CountVectorizer(inputCol='filtered', outputCol='vectors')
count_model = countvec.fit(label0_train)
train = count_model.transform(label0_train)

In [47]:
from pyspark.ml.clustering import LDA

lda = LDA(maxIter=10,k=20,featuresCol='vectors',seed=2)
model = lda.fit(train)

In [48]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType

vocab = count_model.vocabulary

def trans(x):
  return [vocab[i] for i in x]

idx2word = udf(lambda y: trans(y), ArrayType(StringType()))

In [49]:
topics = model.describeTopics(10).withColumn("terms",idx2word("termIndices"))
display(topics)

topic,termIndices,termWeights,terms
0,"List(1, 0, 2, 15, 3, 7, 2682, 5, 137, 81)","List(0.0010836911664018245, 7.246053847748408E-4, 6.960238701490542E-4, 4.2934145975010883E-4, 3.1549689608651615E-4, 2.6107524508118917E-4, 2.556864588178471E-4, 2.504254899292929E-4, 2.4009823968146882E-4, 2.1220520684696114E-4)","List(cat, dog, like, looks, dogs, im, brownie, cats, many, keep)"
1,"List(0, 2, 5, 15, 7, 1, 171, 95, 122, 22)","List(2.4609383813925414E-4, 2.3496217190483349E-4, 1.940620414078316E-4, 1.7135488277154073E-4, 1.486646498112639E-4, 1.4668509942329324E-4, 1.329662979274939E-4, 1.2344544851862718E-4, 1.039618757091071E-4, 1.0383387830179859E-4)","List(dog, like, cats, looks, im, cat, gave, made, kitten, video)"
2,"List(4, 0, 61, 10, 53, 5, 1, 34, 6, 2)","List(4.85000151050246E-4, 3.681647506006394E-4, 3.0652336331612545E-4, 2.875380973149959E-4, 2.3429075444645202E-4, 2.2904835832170847E-4, 2.2656171808059592E-4, 2.256605215662432E-4, 2.2435609319293077E-4, 2.090062243967697E-4)","List(love, dog, hope, know, animals, cats, cat, videos, one, like)"
3,"List(0, 2, 1, 10, 7, 3, 9, 4, 21, 54)","List(0.0035910772957740227, 0.0013737863861962357, 0.001357556013977024, 0.0012229348662404502, 8.246955528033576E-4, 7.837567653044138E-4, 6.931469342851252E-4, 6.651085661713792E-4, 6.187643085770144E-4, 6.030979381659583E-4)","List(dog, like, cat, know, im, dogs, dont, love, people, feel)"
4,"List(0, 1, 5, 2, 6, 4, 19, 51, 91, 6039)","List(7.645693020858014E-4, 6.84344339082834E-4, 5.738714738964939E-4, 4.2541193550508805E-4, 3.594318911644296E-4, 2.6318185934928756E-4, 2.2184165679738546E-4, 2.113573963591227E-4, 2.0107263834237538E-4, 1.9924303837836203E-4)","List(dog, cat, cats, like, one, love, even, little, found, frodo)"
5,"List(3, 2, 6, 0, 12, 44, 323, 2029, 5, 7)","List(3.3200992064487975E-4, 2.8469356741428974E-4, 1.9381313369709423E-4, 1.6565065463659853E-4, 1.505064438403149E-4, 1.2673606580980817E-4, 1.2143562913664653E-4, 1.1981821522952478E-4, 1.1970116228658028E-4, 1.1697450653945935E-4)","List(dogs, like, one, dog, time, cute, brother, account, cats, im)"
6,"List(0, 1, 45, 2, 7, 3, 53, 8, 6, 4949)","List(0.0010217601289505564, 7.93476045221303E-4, 4.406112780851706E-4, 3.8980850339124205E-4, 3.694899459959033E-4, 3.6494420473999446E-4, 3.535436578937345E-4, 3.238541845992745E-4, 3.079767776972558E-4, 3.043498255644679E-4)","List(dog, cat, food, like, im, dogs, animals, get, one, lease)"
7,"List(0, 3, 1, 2, 5, 4, 27, 6, 11, 9)","List(0.0041295584170933704, 0.0038608489256848717, 0.0026360296289947927, 0.002047400418488415, 0.0017881671163066377, 0.0014409864902449748, 0.0011661277103107283, 0.0011131663594914905, 0.0011067603156637144, 0.001101583548190007)","List(dog, dogs, cat, like, cats, love, think, one, got, dont)"
8,"List(3241, 0, 3, 5, 13, 7, 12, 2, 34, 31)","List(6.272396691920975E-4, 6.25476125831048E-4, 4.0298631353893964E-4, 3.3779398969765176E-4, 3.174455766853994E-4, 3.062433735239554E-4, 2.985741656529036E-4, 2.757408986380282E-4, 2.674177635904797E-4, 2.5992034126861945E-4)","List(kss, dog, dogs, cats, really, im, time, like, videos, make)"
9,"List(0, 1, 2, 4, 3, 17, 18, 8, 38, 11)","List(0.0016513476060225523, 0.0014509651605466196, 7.126682073795536E-4, 6.559557367547349E-4, 4.747546057564457E-4, 3.4318587492710286E-4, 3.230425670633077E-4, 3.0903518419565226E-4, 2.981861932816422E-4, 2.892777790679338E-4)","List(dog, cat, like, love, dogs, want, go, get, died, got)"


In [50]:
from pyspark.sql.functions import explode, arrays_zip

topics_words = topics.withColumn("termWithProb", explode(arrays_zip("terms", "termWeights")))
topics_words.createOrReplaceTempView("topics")


In [51]:
%sql

Select topic, termWithProb["terms"] as term, termWithProb["termWeights"] as probability
From topics

-- get words in each topics with probability

topic,term,probability
0,cat,0.0010836911664018245
0,dog,7.246053847748408E-4
0,like,6.960238701490542E-4
0,looks,4.2934145975010883E-4
0,dogs,3.1549689608651615E-4
0,im,2.6107524508118917E-4
0,brownie,2.556864588178471E-4
0,cats,2.504254899292929E-4
0,many,2.4009823968146882E-4
0,keep,2.1220520684696114E-4


#### 4. Identify Creators With Cat And Dog Owners In The Audience

In [53]:

top = spark.sql(
      """Select creator_name, count(*) as num
      From predict
      Where prediction=1
      Group by 1
      Order by 2 DESC
      limit 20""")

display(top)

# top 20 creators with most audience pet owner users

creator_name,num
The Dodo,108175
Brave Wilderness,74341
Robin Seplut,58568
Taylor Nicole Dean,58075
Hope For Paws - Official Rescue Channel,43909
Brian Barczyk,41152
Gohan The Husky,36110
Vet Ranch,32227
Gone to the Snow Dogs,26969
Viktor Larkhill,26764


In [54]:
top.createOrReplaceTempView("top_creators")

In [55]:
%sql

with total as
  (Select creator_name, sum(part_num) as sum_n
  From top_label
  Group by 1)

Select t.creator_name, num/sum_n as fraction
From top_creators as t
Inner join total as total on t.creator_name=total.creator_name
Order by 2 DESC

-- get pet owner users fraction of those top video creators

creator_name,fraction
Zak Georges Dog Training rEvolution,0.6211439588688946
Robin Seplut,0.5025268775687062
Gone to the Snow Dogs,0.48823273833230746
Paws Channel,0.48488169929699904
stacyvlogs,0.47169861839991434
Cole & Marmalade,0.45641524028639996
Gohan The Husky,0.3626375833534185
Think Like A Horse,0.3599107946640478
Viktor Larkhill,0.3505986533574367
Hope For Paws - Official Rescue Channel,0.3438018729055091


In [56]:
%sql

with filter as 
  (Select creator_name, count(*) as num
  From predict
  Group by 1
  Having num>=10000
  ),
  part as 
  (Select pre.creator_name, prediction, count(*) as part_num
  From predict as pre
  Inner join filter as f on pre.creator_name=f.creator_name
  Group by 1,2
  Order by 1)

Select p.creator_name, part_num, part_num/num as fraction
From part as p
Inner join filter as f on p.creator_name=f.creator_name
Where p.prediction = 1
Order by 3 DESC

-- get highest statistically significant percentages of pet owner users with more than 10000 comments

creator_name,part_num,fraction
Zak Georges Dog Training rEvolution,11598,0.6211439588688946
MaxluvsMya,8378,0.5654315988391713
Lennon The Bunny,5802,0.5399218313791178
Robin Seplut,58568,0.5025268775687062
Kitten Academy,9419,0.4911612869583355
Gone to the Snow Dogs,26969,0.48823273833230746
Paws Channel,12829,0.48488169929699904
meow meow,7892,0.47674278120091823
Cat Man Chris,6029,0.4740898010537076
stacyvlogs,17617,0.47169861839991434


In [57]:
# %sql

# with filter as 
#   (Select creator_name, count(*) as num
#   From predict
#   Group by 1
#   Having num>=10000
#   ),
#   part as 
#   (Select t.creator_name, label, count(*) as part_num
#   From predict as t
#   Inner join filter as f on t.creator_name=f.creator_name
#   Group by 1,2
#   Order by 1)

# Select p.creator_name, label, part_num, part_num/num as fraction
# From part as p
# Inner join filter as f on p.creator_name=f.creator_name

#### 5. Analysis and Future work

## Summary

1.Performed analysis on comments for Youtube videos related to pets.  
2.Built data processing pipeline with tokenizer and word2vec to transform comments to feature vectors and identify pet owners based on keywords of comments.  
3.Constructed and tuned classification models based on Logistic Regression, Random Forest and Gradient Boosting to predict if a user is pet owner.  
4.Chose the best model - Random Forest with 93% accuracy to estimate fraction of all users who are pet owners.  
5.Extracted main topics of pet owners using Latent Dirichlet Allocation (LDA).  
6.Identified top 20 attractive video creators to pet owner users based on total number and highest percentage.